In [10]:
import numpy as np
import pandas as pd
import json
from collections import defaultdict, deque

## 1. Data Loading & Preprocessing


In [ ]:
def load_data(file_path):
    df = pd.read_csv(file_path, converters={"Trade_History": json.loads})
    trade_df = df.explode("Trade_History").reset_index(drop=True)
    trade_df = pd.concat(
        [trade_df["Trade_History"].apply(pd.Series), trade_df["Port_IDs"]], axis=1
    )

    # Convert numeric fields
    numeric_cols = ["realizedProfit", "quantity", "price"]
    trade_df[numeric_cols] = trade_df[numeric_cols].apply(
        pd.to_numeric, errors="coerce"
    )

    # Filter valid trades
    trade_df = trade_df.dropna(
        subset=["side", "positionSide", "quantity", "realizedProfit"]
    )
    trade_df["timestamp"] = pd.to_datetime(trade_df["time"], errors="coerce")
    return trade_df.dropna(subset=["times"])

## 2. Position Classification


In [29]:
def classify_positions(df):
    conditions = [
        (df["side"] == "BUY") & (df["positionSide"] == "LONG"),
        (df["side"] == "SELL") & (df["positionSide"] == "LONG"),
        (df["side"] == "SELL") & (df["positionSide"] == "SHORT"),
        (df["side"] == "BUY") & (df["positionSide"] == "SHORT"),
    ]
    choices = ["long_open", "long_close", "short_open", "short_close"]
    df["position_type"] = np.select(conditions, choices, default=None)
    return df.dropna(subset=["position_type"])

## 3. Portfolio Analysis Engine


In [30]:
class PortfolioAnalyzer:
    def _init_(self):
        self.queues = {"long": defaultdict(deque), "short": defaultdict(deque)}
        self.metrics = {
            "realized_profits": [],
            "invested_capital": [],
            "cumulative_pnl": [],
            "returns": [],
        }

    def process_trade(self, trade):
        asset = trade["asset"]
        pos_type = trade["position_type"]
        direction, action = pos_type.split("_")

        if action == "open":
            self.queues[direction][asset].append(
                {
                    "quantity": trade["quantity"],
                    "price": trade["price"],
                    "timestamp": trade["timestamp"],
                }
            )
        else:
            if self.queues[direction][asset]:
                open_trade = self.queues[direction][asset].popleft()
                profit = trade["realizedProfit"]

                # Calculate metrics
                self.metrics["realized_profits"].append(profit)
                self.metrics["invested_capital"].append(open_trade["quantity"])
                self.metrics["returns"].append(profit / open_trade["quantity"])

                # Update cumulative PnL
                current_pnl = (
                    self.metrics["cumulative_pnl"][-1]
                    if self.metrics["cumulative_pnl"]
                    else 0
                )
                self.metrics["cumulative_pnl"].append(current_pnl + profit)

    def get_metrics(self):
        if not self.metrics["realized_profits"]:
            return None

        total_pos = len(self.metrics["realized_profits"])
        win_pos = sum(p > 0 for p in self.metrics["realized_profits"])

        # Sharpe Ratio calculation
        returns = self.metrics["returns"]
        sharpe = np.mean(returns) / np.std(returns) if len(returns) > 1 else 0

        # MDD calculation
        cumulative_pnl = self.metrics["cumulative_pnl"]
        running_max = np.maximum.accumulate(cumulative_pnl)
        drawdowns = running_max - cumulative_pnl
        mdd = np.max(drawdowns) if len(drawdowns) > 0 else 0

        return {
            "ROI": np.sum(self.metrics["realized_profits"])
            / np.sum(self.metrics["invested_capital"])
            * 100,
            "PnL": np.sum(self.metrics["realized_profits"]),
            "Sharpe": sharpe,
            "MDD": mdd,
            "WinRate": win_pos / total_pos,
            "WinPositions": win_pos,
            "TotalPositions": total_pos,
        }

## 4. Main Processing Pipeline


In [31]:
def analyze_portfolios(df):
    results = []
    for port_id, group in df.groupby("Port_ID"):
        analyzer = PortfolioAnalyzer()
        for _, trade in group.sort_values("timestamp").iterrows():
            analyzer.process_trade(trade)

        metrics = analyzer.get_metrics()
        if metrics:
            results.append({"Port_ID": port_id, **metrics})

    return pd.DataFrame(results)

## 5. Ranking System


In [32]:
def rank_portfolios(metrics_df):
    # Normalization
    metrics = ["ROI", "PnL", "Sharpe", "MDD", "WinRate"]
    weights = [0.25, 0.25, 0.2, 0.15, 0.15]

    normalized = metrics_df[metrics].apply(
        lambda x: (x - x.min()) / (x.max() - x.min())
    )
    normalized["MDD"] = 1 - normalized["MDD"]  # Invert MDD

    metrics_df["Score"] = (normalized * weights).sum(axis=1)
    return metrics_df.sort_values("Score", ascending=False)

In [ ]:
# Execution
if __name__ == "__main__":
    # Load and process data
    trade_data = load_data("TRADES_CopyTr_90D_ROI.csv")
    classified_data = classify_positions(trade_data)

    # Analyze portfolios
    portfolio_metrics = analyze_portfolios(classified_data)

    # Rank and get top 20
    ranked_portfolios = rank_portfolios(portfolio_metrics)
    top_20 = ranked_portfolios.head(20)

    # Save results
    portfolio_metrics.to_csv("all_portfolio_metrics.csv", index=False)
    top_20.to_csv("top_20_portfolios.csv", index=False)